In [3]:
# from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup

In [4]:
import numpy as np
import pandas as pd

In [5]:
from transliterate import translit

In [6]:
from time import sleep

In [7]:
import re

In [8]:
# Функция для вычисления расстояния м/у широтой и долготой в километрах
import haversine_dist

### Avito

In [7]:
# Что первично можно выцепить на сайте:
# Город, тип дома, кол-во комнат

#### Получаем аналоги на этой же улице

In [48]:
def get_similar_flats(city, street, n_rooms, house_type):
    base_url = 'https://www.avito.ru/'
    city =  re.findall(r'[a-z- ]+', translit(city.lower(), reversed=True))[0]
    street = street.lower()
    rooms = {
        1: '1-komnatnye',
        2: '2-komnatnye',
        3: '3-komnatnye',
        4: '4-komnatnye'
    }   
    house_types = {
        'Кирпичный': 'kirpichnyy_dom',
        'Панельный': 'panelnyy_dom',
        'Блочный': 'blochnyy_dom',
        'Монолитный': 'monolitnyy_dom',
        'Деревянный': 'derevyannyy_dom'
    }   
    search = base_url + city + '/kvartiry/prodam/vtorichka/' + rooms[n_rooms] + '/' + house_types[house_type] + '?q=' + '+'.join(street.split()) + '&s=104'
    print(search)
    html_doc =  requests.get(search)
    # Проверка на корректность запроса
    if html_doc.status_code == 429:
        print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
    else:
        print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив
    
    # Получаю сайт с каждой квартирой отдельно
    flats_sites = []
    for flat in flats_act:
        flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])
    res = []
    for flat in flats_sites[:5]:
        flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")
        sleep(6.451 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
        flat_one_info = {
                        'floor':None,
                        'all_floors':None,
                        'rooms':None,
                        'all_square':None,
                        'live_square':None,
                        'kitch_square':None,
                        'tall':None,
                        'repair':None,
                        'bathroom':None,
                        'balcony':None,
                        'window':None,
                        'dop_info':None,
                        'house_type':None,
                        'year_built':None,
                        'how_pas_lift':None,
                        'how_grz_lift':None,
                        'in_house':None,
                        'backyard':None,
                        'park':None,
                        'flat_lat':None,
                        'flat_lon':None,
                        'address':None,
                        'price':None,
                        'link':None
                        }

        for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
            label = text.text[:text.text.find(':')].split()[0]
            value = text.text[text.text.find(':') + 1:].split()
            if label == 'Этаж':
                flat_one_info['floor'] = [value[0]]
                flat_one_info['all_floors'] = [value[2]]
            elif label == 'Количество':
                flat_one_info['rooms'] = [value[0]]
            elif label == 'Общая':
                flat_one_info['all_square'] = [value[0]]
            elif label == 'Площадь':
                flat_one_info['live_square'] = [value[0]]
            elif label == 'Жилая':
                flat_one_info['kitchen_square'] = [value[0]]
            elif label == 'Высота':
                flat_one_info['tall'] = [value[0]]   
            elif label == 'Ремонт':
                flat_one_info['repair'] = [value[0]]
            elif label == 'Санузел':
                flat_one_info['bathroom'] = [value[0]]
            elif label == 'Балкон':
                flat_one_info['balcony'] = [value[0]]
            elif label == 'Вид':
                flat_one_info['window'] = [' '.join(value)]
            elif label == 'Дополнительно':
                flat_one_info['dop_info'] = [' '.join(value)]
            elif label == 'Тип':
                flat_one_info['house_type'] = [value[0]]
            elif label == 'Год':
                flat_one_info['year_built'] = [value[0]]
            elif label == 'Пассажирский':
                flat_one_info['how_pas_lift'] = [value[0]]
            elif label == 'Грузовой':
                flat_one_info['how_grz_lift'] = [value[0]]
            elif label == 'В':
                flat_one_info['in_house'] = [' '.join(value)]
            elif label == 'Двор':
                flat_one_info['backyard'] = [' '.join(value)]
            elif label == 'Парковка':
                flat_one_info['park'] = [' '.join(value) ]
            flat_one_info['price'] = [int(flat_i.find('span', class_ = 'js-item-price')['content'])]
        flat_one_info['address'] = [flat_i.find('span', class_ = 'item-address__string').text.strip()]
        flat_one_info['flat_lat'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lat']]
        flat_one_info['flat_lon'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lon']]
        flat_one_info['link'] = [flat]
        df = pd.DataFrame(flat_one_info)
        res.append(df)
        print(' ---------- done! ----------')
        sleep(4.322 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
    return pd.concat(res)

In [45]:
res = get_similar_flats('Хабаровск', 'Гамарника', 1, 'Кирпичный')

https://www.avito.ru/habarovsk/kvartiry/prodam/1-komnatnye/kirpichnyy_dom?q=гамарника&s=104
Кажется, всё ок. Но вот на всякий случай код: 200
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------
 ---------- done! ----------


In [ ]:
res

#### Получаем аналоги в радиусе 1км

In [9]:
def get_similar_flats_1km(address, city, n_rooms, house_type, flat_floor, flat_floors, all_square):
    base_url = 'https://www.avito.ru/'
    
    city =  re.findall(r'[a-z- ]+', translit(city.lower(), reversed=True))[0]
    # Получаю координаты
    coord_url = 'https://www.google.com/maps/search/' + address
    coord_html_doc =  requests.get(coord_url)
    soup = BeautifulSoup(coord_html_doc.text, "html.parser")
    flat_lat = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[0])
    flat_lon = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[1])
    
    rooms = {
        1: '1-komnatnye',
        2: '2-komnatnye',
        3: '3-komnatnye',
        4: '4-komnatnye'
    }   
    house_types = {
        'Кирпичный': 'kirpichnyy_dom',
        'Панельный': 'panelnyy_dom',
        'Блочный': 'blochnyy_dom',
        'Монолитный': 'monolitnyy_dom',
        'Деревянный': 'derevyannyy_dom'
    }   
    search = base_url + city + '/kvartiry/prodam/vtorichka/' + rooms[n_rooms] + '/' + house_types[house_type] + '?s=104'
    html_doc =  requests.get(search)
    # Проверка на корректность запроса
    if html_doc.status_code == 429:
        print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
    else:
        print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив
    pages = soup.find_all('span', class_ = 'pagination-item-JJq_j')  # получаю информацию о количестве страниц с квартирами
    if len(pages) != 0:
        pages = int(pages[-2].text)
        flats_pages = []
        flags_pages.append(flats_act)
        for page in range(2, pages + 1):
            print(page)
            search = base_url + city + '/kvartiry/prodam/vtorichka/' + rooms[n_rooms] + '/' + house_types[house_type] + '?p=' + str(page) + '&s=104'
            html_doc =  requests.get(search)
            # Проверка на корректность запроса
            if html_doc.status_code == 429:
                print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
            else:
                print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
                soup = BeautifulSoup(html_doc.text, "html.parser")
                flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
                flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив
                flats_pages.append(flats_act)
                sleep(5.1252 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
    else:
        pages = 1
    # Проверка на соответствие по площади и этажности
    flats_sites = []
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив
        for flat in flats_act:
            head = flat.a['title']
            floor = int(head[head.find('/') - 1:head.find('/')])
            floors = int(head[head.find('/') + 1:head.find('/') + 2])
            if (floor == 1 and flat_floor == 1) or (floor == floors and flat_floor == flat_floors) or (floor != 1 and flat_floor != 1 and flat_floor != flat_floors and floor != floors):
                square = float(re.match(r'[0-9,]+', head.split(' ')[3]).group().replace(',', '.'))
                if square > all_square*0.8 and square < all_square*1.2:
                    flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])
                else:
                    continue
            else:
                continue
    # Беру 5 аналогов
    res = []
    counter = 1
    while counter <= 5:
        print(len(flats_sites))
        for flat in flats_sites:
            print(flat)
            sleep(6.451 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
            flat_one_info = {
                            'floor':None,
                            'all_floors':None,
                            'rooms':None,
                            'all_square':None,
                            'live_square':None,
                            'kitch_square':None,
                            'tall':None,
                            'repair':None,
                            'bathroom':None,
                            'balcony':None,
                            'window':None,
                            'dop_info':None,
                            'house_type':None,
                            'year_built':None,
                            'how_pas_lift':None,
                            'how_grz_lift':None,
                            'in_house':None,
                            'backyard':None,
                            'park':None,
                            'flat_lat':None,
                            'flat_lon':None,
                            'address':None,
                            'price':None,
                            'link':None
                            }

            flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")

            flat_one_info['flat_lat'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lat']]
            flat_one_info['flat_lon'] = [flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lon']]
            
            
            
            if haversine_dist.haversine(float(flat_one_info['flat_lon'][0]), float(flat_one_info['flat_lat'][0]), flat_lon, flat_lat) >= 1:
                print('skipped!')
                continue
            else:
                flat_one_info['price'] = [int(flat_i.find('span', class_ = 'js-item-price')['content'])]
                flat_one_info['address'] = [flat_i.find('span', class_ = 'item-address__string').text.strip()]
                flat_one_info['link'] = [flat]
                for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
                    label = text.text[:text.text.find(':')].split()[0]
                    value = text.text[text.text.find(':') + 1:].split()
                    if label == 'Этаж':
                        flat_one_info['floor'] = [value[0]]
                        flat_one_info['all_floors'] = [value[2]]
                    elif label == 'Количество':
                        flat_one_info['rooms'] = [value[0]]
                    elif label == 'Общая':
                        flat_one_info['all_square'] = [value[0]]
                    elif label == 'Площадь':
                        flat_one_info['live_square'] = [value[0]]
                    elif label == 'Жилая':
                        flat_one_info['kitchen_square'] = [value[0]]
                    elif label == 'Высота':
                        flat_one_info['tall'] = [value[0]]   
                    elif label == 'Ремонт':
                        flat_one_info['repair'] = [value[0]]
                    elif label == 'Санузел':
                        flat_one_info['bathroom'] = [value[0]]
                    elif label == 'Балкон':
                        flat_one_info['balcony'] = [value[0]]
                    elif label == 'Вид':
                        flat_one_info['window'] = [' '.join(value)]
                    elif label == 'Дополнительно':
                        flat_one_info['dop_info'] = [' '.join(value)]
                    elif label == 'Тип':
                        flat_one_info['house_type'] = [value[0]]
                    elif label == 'Год':
                        flat_one_info['year_built'] = [value[0]]
                    elif label == 'Пассажирский':
                        flat_one_info['how_pas_lift'] = [value[0]]
                    elif label == 'Грузовой':
                        flat_one_info['how_grz_lift'] = [value[0]]
                    elif label == 'В':
                        flat_one_info['in_house'] = [' '.join(value)]
                    elif label == 'Двор':
                        flat_one_info['backyard'] = [' '.join(value)]
                    elif label == 'Парковка':
                        flat_one_info['park'] = [' '.join(value)]
            df = pd.DataFrame(flat_one_info)
            res.append(df)
            print(' ---------- done! ----------')
            counter += 1
            sleep(4.322 + np.random.normal(1,2,1)[0])  # добавляю зашумление времени 
    return pd.concat(res)

In [10]:
res = get_similar_flats_1km('Хабаровский край, г. Хабаровск, ул. Гамарника, д. 80А', 'Хабаровск', 1, 'Кирпичный', 1, 5, 28.6)

https://www.avito.ru/habarovsk/kvartiry/prodam/vtorichka/1-komnatnye/kirpichnyy_dom?s=104
Кажется, всё ок. Но вот на всякий случай код: 200
5
https://www.avito.ru/habarovsk/kvartiry/1-k._kvartira_297_m_15_et._2248937781


KeyboardInterrupt: 

In [ ]:
pagination-item-JJq_j

In [32]:
html_doc =  requests.get('https://www.avito.ru/habarovsk/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?f=ASgBAQICAUSSA8YQA0DkBxT4UeYHFIxSyggUgFk&p=2&s=104')
# Проверка на корректность запроса
if html_doc.status_code == 429:
    print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
else:
    print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")

Кажется, всё ок. Но вот на всякий случай код: 200


In [33]:
# Нахожу все подходящие (не рекомендации) квартиры на этой странице
flats = soup.find_all('span', class_ = 'pagination-item-JJq_j')  # получаю список квартир вместе с рекламой и рекомендациями

In [36]:
int(flats[-2].text)

8

#### Получение координат из адреса по Гугл-картам

In [44]:
url = 'https://www.google.com/maps/search/' + 'Пензенская обл, г Кузнецк, ул 60-летия ВЛКСМ, д 8'

In [47]:
html_doc =  requests.get(url)

# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")

In [88]:
source_lat = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[0])
source_lon = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[1])

In [50]:
html_doc =  requests.get('https://www.avito.ru/habarovsk/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?f=ASgBAQICAUSSA8YQA0DkBxT4UeYHFIxSyggUgFk&s=104')
# Проверка на корректность запроса
if html_doc.status_code == 429:
    print('Сервер забанил за подозрительную активность! Разбанит примерно через час')
else:
    print('Кажется, всё ок. Но вот на всякий случай код: {}'.format(html_doc.status_code))
# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")
# Нахожу все подходящие (не рекомендации) квартиры на этой странице
flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив

Кажется, всё ок. Но вот на всякий случай код: 200


In [52]:
len(flats_act)

59

In [106]:
flat_floor = 1
flat_floors = 5

In [112]:
# Получаю сайт с каждой квартирой отдельно
for flat in flats_act:
    head = flat.a['title']
    floor = int(head[head.find('/') - 1:head.find('/')])
    floors = int(head[head.find('/') + 1:head.find('/') + 2])
    if (floor == 1 and flat_floor == 1) or (floor == floors and flat_floor == flat_floors) or (floor != 1 and flat_floor != 1 and flat_floor != flat_floors and floor != floors):
        square = float(re.match(r'[0-9,]+', head.split(' ')[3]).group().replace(',', '.'))
        if square > 28.6*0.8 and square < 28.6*1.2:
            print(head)
        else:
            continue
    else:
        continue

Объявление «1-к. квартира, 29,7 м², 1/5 эт.» 11 фотографий
Объявление «1-к. квартира, 31 м², 1/5 эт.» 5 фотографий
Объявление «1-к. квартира, 24 м², 1/5 эт.» 13 фотографий
Объявление «1-к. квартира, 29,5 м², 1/5 эт.» 4 фотографии
Объявление «1-к. квартира, 30,5 м², 1/5 эт.» 13 фотографий
